In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features(select) -> pd.DataFrame:
    return batch_load_sql(select)

In [3]:
posts_df = pd.read_csv('clusterized_post_df.csv')

In [4]:
posts_df.drop('ada_embedding', axis=1, inplace=True)

## Find total post likes and like rate

In [6]:
select = "SELECT COUNT(*), post_id FROM feed_data WHERE action = 'like' GROUP BY post_id"
posts_likes = load_features(select)
posts_likes = posts_likes.set_index('post_id')

In [7]:
posts_df['total_likes'] = posts_df['post_id'].map(posts_likes['count'])

In [8]:
select = "SELECT COUNT(*), post_id FROM feed_data GROUP BY post_id"
posts_actions = load_features(select)

In [9]:
posts_actions = posts_actions.set_index('post_id')

In [10]:
posts_df['like_rate'] = posts_df['post_id'].map(posts_likes['count'] / posts_actions['count'])

In [11]:
posts_df['like_rate'] = posts_df['like_rate'].fillna(posts_df['like_rate'].mean())
posts_df['total_likes'] = posts_df['total_likes'].fillna(posts_df['total_likes'].mean())

## Find post avarege liking age

In [18]:
select = """
            SELECT AVG(age), pt.post_id
            FROM feed_data as fd
            JOIN user_data as ud
            ON fd.user_id = ud.user_id
            JOIN post_text_df as pt
            ON fd.post_id = pt.post_id
            WHERE action = 'like'
            GROUP BY pt.post_id
         """
age_likes = load_features(select)

In [20]:
age_likes = age_likes.set_index('post_id')

In [21]:
posts_df['avg_liking_age'] = posts_df['post_id'].map(age_likes['avg'])

In [23]:
posts_df['avg_liking_age'] = posts_df['avg_liking_age'].fillna(posts_df['avg_liking_age'].mean())

In [27]:
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
posts_df.to_sql('b_shramko_posts_features_lesson_22', con=engine, if_exists='replace', index=False)

23

In [26]:
posts_df.to_csv('clusterized_post_df.csv', index=False)

In [25]:
posts_df

,post_id,text,topic,average_sentence_length,post_len,tsne-2d-one,tsne-2d-two,cluster,total_likes,like_rate,avg_liking_age
0,1,UK economy facing major risks\n\nThe UK manufa...,business,19.117647,1967,-54.696510,-3.841329,19,1067.0,0.111436,29.099344
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,23.631579,2701,-51.182384,-20.905855,19,637.0,0.078333,31.910518
2,3,Asian quake hits European shares\n\nShares in ...,business,10.865385,3408,-49.081220,-15.098360,19,1122.0,0.117684,29.262923
3,4,India power shares jump on debut\n\nShares in ...,business,15.818182,1026,-43.553980,-14.994580,4,1171.0,0.125523,29.084543
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,21.428571,889,-30.916187,-18.663040,4,1153.0,0.118426,29.479618
...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,14.454545,803,23.089860,8.660326,3,2619.0,0.133903,22.554792
7019,7316,I give this movie 2 stars purely because of it...,movie,14.090909,800,44.205605,13.038532,27,677.0,0.093392,31.441654
7020,7317,I cant believe this film was allowed to be mad...,movie,9.416667,636,55.726883,20.266338,23,731.0,0.097027,31.885089
7021,7318,The version I saw of this film was the Blockbu...,movie,13.000000,728,20.359941,7.068708,3,680.0,0.091092,32.423529


## Find favorite topic for each user

In [50]:
select = "SELECT user_id, post_id FROM feed_data WHERE action = 'like'"
users_likes = load_features(select)

In [54]:
users_likes_posts = pd.merge(users_likes, posts_df[['post_id', 'topic']], on='post_id', how='left')

In [55]:
popular_topics_per_user = users_likes_posts.groupby(['user_id', 'topic']).size().reset_index(name='topic_count')

most_popular_topic_per_user = popular_topics_per_user.loc[popular_topics_per_user.groupby('user_id')['topic_count'].idxmax()]

most_popular_topic_per_user = most_popular_topic_per_user.set_index('user_id')

In [56]:
select = "SELECT * from user_data"
users_df = load_features(select)

In [57]:
users_df['favorite_topic'] = users_df['user_id'].map(most_popular_topic_per_user['topic']).fillna('unknown')

In [64]:
users_df['favorite_topic'].value_counts()

favorite_topic
movie            129667
covid             24896
sport              5148
business           1620
politics           1026
entertainment       825
tech                 20
unknown               3
Name: count, dtype: int64

In [63]:
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )

users_df.to_sql('b_shramko_users_features_lesson_22', con=engine, if_exists='replace', index=False)

205

In [62]:
users_df.to_csv('users.csv', index=False)